# Demo Chroma

Demo 1

In [33]:
!pip install chromadb

In [34]:
# Get the Chroma Client
import chromadb
chroma_client = chromadb.Client()

In [37]:
#Create a collection
collection = chroma_client.create_collection(name="collection")

In [39]:
# Add some text documents to the collection
collection.add(
    documents=["Đây là một câu nói.", "Đây là một video"],
    metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

In [40]:
results = collection.query(
    query_texts=["Đây là một câu hỏi"],
    n_results=2,
#    include=['distances','metadatas','embeddings', 'documents']

)

In [41]:
results

{'ids': [['id1', 'id2']],
 'distances': [[0.1728145182132721, 0.5370555520057678]],
 'metadatas': [[{'source': 'my_source'}, {'source': 'my_source'}]],
 'embeddings': None,
 'documents': [['Đây là một câu nói.', 'Đây là một video']],
 'uris': None,
 'data': None}

Demo2

In [42]:
import csv

# Load sample data (a restaurant menu of items)
with open('menu_items.csv') as file:
    lines = csv.reader(file)

    # Store the name of the menu items in this array. In Chroma, a "document" is a string i.e. name, sentence, paragraph, etc.
    documents = []

    # Store the corresponding menu item IDs in this array.
    metadatas = []

    # Each "document" needs a unique ID. This is like the primary key of a relational database. We'll start at 1 and increment from there.
    ids = []
    id = 1

    # Loop thru each line and populate the 3 arrays.
    for i, line in enumerate(lines):
        if i==0:
            # Skip the first row (the column headers)
            continue

        documents.append(line[1])
        metadatas.append({"item_id": line[0]})
        ids.append(str(id))
        id+=1

In [43]:
# Install sentence transformers
# This is used to convert text to vector embeddings. In other words, it converts text to a bunch of numbers that represent the 'meaning' of the text.
%pip install -U sentence-transformers

In [44]:
import chromadb
from chromadb.utils import embedding_functions

# Instantiate chromadb instance. Data is stored on disk (a folder named 'my_vectordb' will be created in the same folder as this file).
chroma_client = chromadb.PersistentClient(path="my_vectordb")

In [45]:
# Select the embedding model to use.
# List of model names can be found here https://www.sbert.net/docs/pretrained_models.html
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")

# Create the collection, aka vector database. Or, if database already exist, then use it. Specify the model that we want to use to do the embedding.
collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=sentence_transformer_ef)

In [46]:
# Add all the data to the vector database. ChromaDB automatically converts and stores the text as vector embeddings. This may take a few minutes.
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [47]:
# Query the vector database

# Query mispelled word: 'vermiceli'. Expect to find the correctly spelled 'vermicelli' item
results = collection.query(
    query_texts=["vermiceli"],
    n_results=5,
    include=['documents', 'distances', 'metadatas']
)
print(results['documents'])

[['Melon and Vermicelli', 'Vegetable Tempura', 'Veggie Lee Secret Drink', 'San Pellegrino', 'Assorted Vegetables']]
